In [ ]:
from dotenv import load_dotenv
import os
from langchain.document_loaders import CSVLoader
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings

from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

In [ ]:
load_dotenv()

In [ ]:
loader = CSVLoader(file_path="./e-commerce-data.csv")
documents = loader.load()

# **all-MiniLM-L6-v2** model

In [5]:
# practice
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

db = Chroma.from_documents(documents, embedding_function)
query = "How many quantity of the wallet is left?"
docs = db.similarity_search(query)
print(docs[0].page_content)


Name: TRAVEL CARD WALLET PANTRY
Quantity: 24
UnitPrice: 0.42


In [11]:
query = "I am thirsty so want to buy some drinks."
docs = db.similarity_search(query)
print(docs[0].page_content)

Name: GIN AND TONIC MUG
Quantity: 24
UnitPrice: 1.25


# LangChain Expression with Chroma DB CSV (RAG) - **OpenAIEmbeddings** model

In [13]:
embeddings_ = OpenAIEmbeddings()

loader = CSVLoader(file_path="./e-commerce-data-short2.csv")
documents = loader.load()

db = Chroma.from_documents(documents, embeddings_)
retriever = db.as_retriever()

template = """
You are the best online shopping mall seller.
Answer the question based only on the following context and basically recommend the product with unit price:
{context}

Question: {question}
"""
template = """
Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI()

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)


InvalidDimensionException: Embedding dimension 1536 does not match collection dimensionality 384

In [ ]:
print(chain.invoke("I am thirsty so want to buy some drinks."))
print(chain.invoke("I need a hanger to hang."))
print(chain.invoke("Soon my friend's birthday."))

# Question list
- I am thirsty so want to buy some drinks.
- Is there any wallet?
- Soon my friend's birthday.